# **Customer Churn Prediction Notebook**
This notebook demonstrates how to load the trained Random Forest model and make predictions on new customer data.

In [ ]:
import pandas as pd
import joblib
import os

# Define paths
MODEL_PATH = '../models/rf_model.pkl'
DATA_PATH = '../data/processed_churn_data.parquet'

# Load the model
if os.path.exists(MODEL_PATH):
    model = joblib.load(MODEL_PATH)
    print("Model loaded successfully!")
else:
    print(f"Error: Model not found at {MODEL_PATH}. Please run the training notebook first.")